In [1]:
#Full Code
#******************************************************
def load_faces():
    #with open('dataset_faces.dat', 'rb') as f:
    with open('mca_faces_dataset.dat','rb') as f:
        all_face_encodings = pickle.load(f)

# Grab the list of names and the list of encodings
    known_faces_names = list(all_face_encodings.keys())
    known_faces_encodings = np.array(list(all_face_encodings.values()))

   # print("The Databases contains metadata of these faces" +  str(known_faces_names))
    name = known_faces_names[0].split('_')
    
    return [known_faces_names,known_faces_encodings,name]

#******************************************************
#Sending an email
def send_mail():
    tt = {
      'Monday':{10:'d',11:'m',12:'m',2:'d'},
      'Tuesday':{10:'m',11:'h',12:'d',2:'d'},
      'Wednesday':{10:'a',11:'a',12:'m',2:'d',3:'h'},
      'Thursday':{10:'d',11:'a',12:'a',2:'d',6:'d'},
      'Friday':{10:'h',11:'h',12:'d',1:'a',2:'d'},
      'Sunday':{2:'d'}
     }
    emails = {'d':'dinesh_sir_mail','a':'ashish_sir_mail','m':'minesh_sir_mail','h':'harshali_maam_mail'}
    day = datetime.datetime.today().strftime('%A')
    print(day)
    time_12h = int(round(float(time[:2]),2))
    if time_12 >= 13:
        time_12 -=12
    print(time)
    email_id = tt[str(day)][time_12h]
    print(str(emails[email_id]) + ' Will get the mail')
    print('Mail has been delivered')

"""
    with open('login_data.dat', 'rb') as f:
        login_data = pickle.load(f)
    print(login_data["username"])
    # list of email_id to send the mail 
    send_email = ["your_mail_id"]
    subject = "Attendance"
    message = "Please download the attendance for your lecture"
    file_location = "Attendance.xlsx"
    msg = MIMEMultipart()
    msg['From'] = login_data["username"]
    msg['To'] = login_data["password"]
    msg['Subject'] = subject
    msg.attach(MIMEText(message, 'plain'))
    li = ["your_email.com"]
    # Setup the attachment
    filename = os.path.basename(file_location)
    attachment = open(file_location, "rb")
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(attachment.read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', "attachment; filename= %s" % filename)

    # Attach the attachment to the MIMEMultipart object
    msg.attach(part)

    for i in range(len(li)): 
        s = smtplib.SMTP('smtp.gmail.com', port) 
        s.starttls() 
        s.login(login_data["username"], login_data["password"]) 
        text = msg.as_string()
        s.sendmail("your_mail.com", send_email[i], text)
        print('The mail will be sent to '+  ) 
        print("The mail has been delivered")
        #message = "New Message sent from python IDE"
        #s.sendmail("username.com", send_email[i], message)

s.quit()
"""
#******************************************************

import openpyxl
import face_recognition
import cv2
import numpy as np
import datetime
import pickle
import smtplib
import os
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(0)


# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
(known_faces_names,known_faces_encodings,name) = load_faces()
wb = openpyxl.load_workbook('Attendance.xlsx')
ws= wb['Attend']

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    #rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(small_frame)
        face_encodings = face_recognition.face_encodings(small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_faces_encodings, face_encoding)
            name = "Unknown"
            face_names.append(name)

            # # If a match was found in known_face_encodings, just use the first one.
            if True in matches:
                face_names.clear()
                first_match_index = matches.index(True)
                name = known_faces_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            #face_distances = face_recognition.face_distance(known_faces_encodings, face_encoding)
            #best_match_index = np.argmin(face_distances)
            
            if matches[first_match_index]:
                name = known_faces_names[first_match_index]
                name = name.split(sep='_')

                face_names.append(name[0])
                aid ='E' + str(name[1])
                ws[aid] = 'Present'
        

   # process_this_frame = not process_this_frame


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
now = datetime.datetime.now()
date = now.strftime("%d-%m-%Y")
time = now.strftime("%H:%M")
ws["D8"] = 'Date: ' + str(date)
ws["E8"] = 'Time: ' + str(time)
video_capture.release()
cv2.destroyAllWindows()
wb.save('Attendance_New.xlsx')
send_mail()

#******************************************************

Sunday


UnboundLocalError: local variable 'time_12' referenced before assignment

In [ ]:
q